In [1]:
import h3
import pandas as pd
import keplergl as KeplerGl
import requests
import json

In [2]:
url_1 = 'https://nominatim.openstreetmap.org/search.php'
params_2 = {
'q':f'amsterdam', 
'polygon_geojson':1,
'format':'geojson'
}

geo_object = requests.get(url_1, params_2)
geo_json_object = json.loads(geo_object.content)
geo_feature = geo_json_object.get('features')[0].get('geometry')


In [4]:
#print(geo_feature)

In [10]:
# geo_feature gives back a multiPolygon
# We can't work with it so we manipulate the dict

manipulated_geo_feature = {
    'type': 'Polygon',
    'coordinates': geo_feature["coordinates"][0]
}


In [11]:
ROME_h3_from_polyfill = h3.polyfill_geojson(manipulated_geo_feature, res=9)
ROME_h3_list =[]
for i in ROME_h3_from_polyfill:
    ROME_h3_list.append(i)

    # Make DF for Munich for POI
ROME_dict = {
    'city': 'ROME',
    'h3_code': ROME_h3_list
}
ROME_poi_df = pd.DataFrame(ROME_dict)

In [12]:
h3_All = h3.polyfill_geojson(manipulated_geo_feature, res=9)
df_rome = pd.DataFrame(h3_All)
df_rome.to_csv('AMSTERDAM_all.csv', sep=',')

### #4 Get ROME all POI

In [13]:
#find the center of every h3_code

ROME_geo_center_list = []

for i in ROME_h3_list:
    k = list(h3.h3_to_geo(i))
    ROME_geo_center_list.append(k)

ROME_geo_center_list_string = []

def make_string_geo():
    for i in ROME_geo_center_list:
        string = str(i[0]) + ',' + str(i[1])
        ROME_geo_center_list_string.append(string)
make_string_geo()

In [14]:
api_key= 'Z1XB-w87ksYmy5gJnj4gfLPJUJRkiyyPDLEtyjnMYy0'
# url_range = f'https://browse.search.hereapi.com/v1/browse?at={GEO}&in=circle:{GEO};r=700&limit=100&apiKey={api_key}'

ROME_title_list = []
ROME_cat_id_list = []
ROME_cat_name_list = []
ROME_h3_source_list = []


count = 0

for index, GEO in enumerate(ROME_geo_center_list_string):
    resp = requests.get(url = f'https://browse.search.hereapi.com/v1/browse?at={GEO}&in=circle:{GEO};r=200&limit=100&apiKey={api_key}')
    data= resp.json()

    for i in data['items']:
        if 'categories' not in i:
            continue
        else:
            ROME_title_list.append(str(i['title']))
            ROME_cat_id_list.append(str(i['categories'][0]['id']))
            ROME_cat_name_list.append(str(i['categories'][0]['name']))
            ROME_h3_source_list.append(ROME_h3_list[index])
            count +=1
print('POI`s found: ' + str(count))

POI`s found: 60948


In [15]:
ROME_here_dict = { 'title': ROME_title_list, 
                'cat_id': ROME_cat_id_list, 
                'cat_name': ROME_cat_name_list, 
                'h3_source': ROME_h3_source_list} 
ROME_poi_df = pd.DataFrame(ROME_here_dict)

In [16]:
def make_cat_def(row):
    if row['cat_id'][:1] == '1':
        return 'Eat & Drink'
    elif row['cat_id'][:1] == '2':
        return 'Going Out-Entertainment'
    elif row['cat_id'][:1] == '3':
        return 'Sights and Museums'
    elif row['cat_id'][:1] == '4':
        return 'Transport'
    elif row['cat_id'][:1] == '5':
        return 'Accommodations'
    elif row['cat_id'][:1] == '6':
        return 'Shopping'
    elif row['cat_id'][:1] == '7':
        return 'Business and Services'
    elif row['cat_id'][:1] == '8':
        return 'Facilities'
    elif row['cat_id'][:1] == '9':
        return 'Areas and Buildings'

In [17]:
ROME_poi_df['cat_definition'] = ROME_poi_df.apply(lambda row : make_cat_def(row), axis=1)

ROME_poi_df = ROME_poi_df[[ 'h3_source',
                            'cat_definition', 
                            'cat_name',
                            'title'
                ]]
ROME_poi_df.to_csv('AMSTERDAM_poi_df.csv', sep=',')